In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import World
from src import PointLight
from src import ipynb_display_image
from src import Resolution
from src import Camera
from src import Scene

import numpy as np
import math
import random
from src.geometry.hittable import Hittable
from src.geometry.hit_point import HitPoint
from src.geometry.ray import Ray
from src.math import Vertex, Vector
from dataclasses import dataclass

# You can create your own geometry objects by implementing the Hittable interface.
- Here is an example of a Torus (doughnut shape) implementation and its usage in a simple scene.
- hittable interface requires implementing `intersect`, `random_point`, and `normal_at` methods.

In [ ]:
@dataclass
class Torus(Hittable):
    """
    A torus object. Donut shape defined by a major radius (distance from center to tube center) and a minor radius (tube radius). Uses as example of custom Hittable implementation. in jupyter notebook.
    The torus is centered at 'center' vertex and l
    """
    center: Vertex
    radius_major: float  # Major radius (distance from center to tube center)
    radius_tube: float  # Minor radius (tube radius)
    material: any


    def normal_at(self, point: Vertex) -> Vector:
        local_hit = point - self.center
        nx = 4 * local_hit.x * (local_hit.x**2 + local_hit.y**2 + local_hit.z**2 - self.radius_major**2 - self.radius_tube**2)
        ny = 4 * local_hit.y * (local_hit.x**2 + local_hit.y**2 + local_hit.z**2 - self.radius_major**2 - self.radius_tube**2) + 8 * self.radius_major**2 * local_hit.y
        nz = 4 * local_hit.z * (local_hit.x**2 + local_hit.y**2 + local_hit.z**2 - self.radius_major**2 - self.radius_tube**2)
        return Vector(nx, ny, nz).normalize()


    def intersect(self, ray: Ray, t_min=1e-3, t_max=float('inf')) -> HitPoint | None:
        ray_origin = ray.origin - self.center
        rd = ray.direction

        # coefficients for quartic equation
        ox, oy, oz = ray_origin.x, ray_origin.y, ray_origin.z
        dx, dy, dz = rd.x, rd.y, rd.z

        sum_d_sq = dx*dx + dy*dy + dz*dz
        e = ox*ox + oy*oy + oz*oz - self.radius_major*self.radius_major - self.radius_tube*self.radius_tube
        f = ox*dx + oy*dy + oz*dz
        four_R2 = 4.0 * self.radius_major * self.radius_major

        coeffs = [
            sum_d_sq*sum_d_sq,
            4.0 * sum_d_sq * f,
            2.0 * sum_d_sq * e + 4.0 * f*f + four_R2 * dy*dy,
            4.0 * f*e + 2.0 * four_R2 * oy * dy,
            e*e - four_R2 * (self.radius_tube*self.radius_tube - oy*oy)
        ]

        roots = np.roots(coeffs)
        roots = np.real(roots[np.isreal(roots)])
        roots = [r for r in roots if t_min < r < t_max]

        if not roots:
            return None

        t = min(roots)
        hit_point = ray.point_at(t)

        normal = self.normal_at(hit_point)

        return HitPoint(
            point=hit_point,
            normal=normal,
            material=self.material,
            dist=t,
            ray_dir=ray.direction
        )


    def random_point(self) -> Vertex:
        theta = random.uniform(0, 2*math.pi)
        phi = random.uniform(0, 2*math.pi)
        x = (self.radius_major + self.radius_tube * math.cos(phi)) * math.cos(theta)
        y = self.radius_tube * math.sin(phi)
        z = (self.radius_major + self.radius_tube * math.cos(phi)) * math.sin(theta)
        return Vertex(self.center.x + x, self.center.y + y, self.center.z + z)


In [ ]:
red = Color.custom_rgb(255, 0, 0)

mirror_spec = Color.custom_rgb(255, 255, 255)
glossy_blue = PhongMaterial(
    name="glossy_blue",
    base_color=Color.custom_rgb(0, 0, 255),
    spec_color=mirror_spec,
    shininess=100,
    ior=1.5,
    reflectivity=0.4,
    transparency=0.0
)

# objects
torus = Torus(center=Vertex(0, 0.2, -5), radius_major=1.5, radius_tube=0.8, material=glossy_blue)

# world
world = World()
world.add(torus)

# lights
point_light = PointLight(position=Vertex(5, 5, 0), intensity=2000.0, falloff=0.01)

# scene setup
camera = Camera(
    fov = 60,
    resolution = Resolution.R360p,
    origin = Vertex(0, 4, 0),
    direction = Vector(0, -0.75, -1),
)

scene = Scene(
    camera = camera,
    world = world,
    lights = [point_light],
    skybox_path = "./skybox/shanghai_4k.hdr"
)

img_path = scene.render_preview()

ipynb_display_image(img_path)

#todo not working with multithreaded yet
# img_path = scene.render_multithreaded()

# ipynb_display_image(img_path)